
## 1. The object we want to compute

We start from a single probabilistic quantity:

$$
\mathbb P^{\mathbb Q}\!\left(
S_{t_i} > B,\;
S_{t_1} < A,\;
\ldots,\;
S_{t_{i-1}} < A
\right).
$$

This is **one event**, but it involves **several random variables observed at different dates**:
$S_{t_1},\ldots,S_{t_i}$.
Because more than one random variable is involved, this is a **joint probability**.

## 2. Reminder: probability for a single continuous variable

For a single continuous random variable $X$ with density $f_X$,
the probability of an event is obtained by integrating the density over the region
where the event holds:

$$
\mathbb P(a < X < b)
=
\int_a^b f_X(x)\,dx.
$$

The density tells us how probability mass is distributed along the real line.

## 3. Extension to several random variables (joint distributions)

When we consider a vector of continuous random variables

$$
X = (X_1,\ldots,X_i),
$$

its behavior is described by a **joint density function**
$f_X(x_1,\ldots,x_i)$.

The fundamental rule is:

> The probability that $X$ falls inside a region
> $D \subset \mathbb R^i$
> is obtained by integrating the joint density over that region.

Mathematically,

$$
\mathbb P(X \in D) = \int_D f_X(x)\,dx.
$$

This is the multidimensional analogue of the one-dimensional formula.

## 4. From prices to Gaussian variables

Under the Black–Scholes–Merton model,

$$
S_{t_k}
=
S_0 \exp\!\left(
(r-q-\tfrac12\sigma^2)t_k + \sigma W_{t_k}
\right).
$$

Since $W_{t_k} = \sqrt{t_k}\,Z_k$ with $Z_k \sim \mathcal N(0,1)$, we can write

$$
S_{t_k}
=
S_0 \exp\!\left(
(r-q-\tfrac12\sigma^2)t_k + \sigma\sqrt{t_k}\,Z_k
\right).
$$

The vector $(Z_1,\ldots,Z_i)$ is **jointly Gaussian**,
meaning it admits a multivariate normal density.

## 5. Translating the event into inequalities on $Z$

Each condition on $S_{t_k}$ becomes a condition on $Z_k$:

- **Survival before autocall** ($j<i$):
$$
S_{t_j} < A
\;\Longleftrightarrow\;
Z_j < a_j.
$$

- **Coupon payment at $t_i$**:
$$
S_{t_i} > B
\;\Longleftrightarrow\;
Z_i > b_i.
$$

Therefore, the original probability becomes

$$
\mathbb P\!\left(
Z_1<a_1,\ldots,Z_{i-1}<a_{i-1},\;
Z_i>b_i
\right).
$$

## 6. Identifying the integration region

The event above defines the region

$$
D
=
(-\infty,a_1]
\times
\cdots
\times
(-\infty,a_{i-1}]
\times
[b_i,+\infty)
\;\subset\;
\mathbb R^i.
$$

Geometrically, this is a rectangular (but unbounded) region in $i$-dimensional space.

## 7. Writing the probability as an integral (detailed explanation)

Because $(Z_1,\ldots,Z_i)$ has a joint density $\varphi_i(z;\Sigma)$,
the probability of any event is **defined** as

$$
\mathbb P(Z \in D) = \int_D \varphi_i(z;\Sigma)\,dz.
$$

Here, $\int_D$ means:
*integrate the joint density over all points $z=(z_1,\ldots,z_i)$ that belong to the region $D$.*

Since $D$ is a Cartesian product of intervals,
multivariable calculus (Fubini–Tonelli theorem) allows us to rewrite this integral
as an **iterated integral**, integrating one variable at a time over its own bounds:

$$
\mathbb P(Z \in D)
=
\int_{-\infty}^{a_1}
\cdots
\int_{-\infty}^{a_{i-1}}
\int_{b_i}^{+\infty}
\varphi_i(z_1,\ldots,z_i;\Sigma)
\,dz_i\cdots dz_1.
$$

No new probability argument is introduced here:
this is purely a change from abstract notation (“integrate over $D$”)
to explicit integration limits.

## 8. Interpretation for autocall pricing

This integral represents the probability that:

- the underlying has remained below the autocall threshold
  at all previous observation dates,
- and finishes above the coupon barrier at the current date.

The formula is exact under Black–Scholes–Merton,
but it does not admit a simple closed form in general.
This is why Monte Carlo simulation is the natural numerical tool
for pricing autocall coupons.


## 9. Why Monte Carlo?

We have written the probability as an integral over a region $D$ in $\mathbb R^n$:

$$
P = \int_D \varphi(z)\,dz.
$$

### Interpretation as an Expectation
Using the indicator function $\mathbb{1}_{Z \in D}$ (which is 1 if $Z \in D$ and 0 otherwise),
we can rewrite this integral as an expectation:

$$
P = \int_{\mathbb R^n} \mathbb{1}_{z \in D} \, \varphi(z)\,dz = \mathbb E[\mathbb{1}_{Z \in D}].
$$

### The Curse of Dimensionality
Why can't we just use standard calculus methods (like Riemann sums or grids) to solve this integral?
For an autocallable product with $n$ observation dates (e.g., observing the stock price every 6 months for 5 years means $n=10$), the integral effectively lives in $n$-dimensional space.

*   If $n=1$ or $2$, we could use standard numerical integration (like drawing a grid on a piece of paper).
*   However, autocalls often have $n=5, 10,$ or even higher.

Standard grid-based methods suffer from the **curse of dimensionality**. If you need $k$ grid points per dimension to get a good answer:
*   In 1 dimension, you need $k$ points.
*   In 2 dimensions, you need $k^2$ points.
*   In 10 dimensions, you need $k^{10}$ points.

If $k=100$, then $100^{10}$ is an astronomically large number of calculations, making grid methods impossible for high-dimensional financial products.

The **Central Limit Theorem** states that for a sufficiently large number of samples $M$, the distribution of this sample mean approaches a normal distribution. The standard deviation of this sample mean, also known as the **standard error (SE)**, is given by:

$$
SE = \frac{\sigma}{\sqrt{M}},
$$
where $\sigma$ is the standard deviation of the individual random variable $\mathbb{1}_{Z \in D}$.

Crucially, the standard error of this estimate scales as $O(1/\sqrt{M})$, **independently of the dimension $n$**. This is the key insight that allows Monte Carlo methods to overcome the "curse of dimensionality." While deterministic numerical integration methods (like grid-based quadratures) have an error that typically scales as $O(1/N^{c/n})$ where $N$ is the total number of grid points and $n$ is the dimension (leading to $N=k^n$ for $k$ points per dimension), Monte Carlo's error convergence rate does not degrade with increasing dimension. Although the convergence rate of $O(1/\sqrt{M})$ is slower than for low-dimensional deterministic methods, its independence from $n$ makes it the only practical method for integrals in high dimensions ($n \ge 4$). This makes Monte Carlo suitable for high-dimensional financial products like autocalls.

### Algorithm Summary
1. Simulate $M$ paths of the asset prices $(S_{t_1}, \ldots, S_{t_n})$.
2. For each path, check if the autocall/coupon conditions are met (is $Z \in D$?).
3. Average the payoffs (discounted back to today).


## 10. Algorithm Formulation

To solve the integral numerically, the program implements the following mathematical scheme:

### 10.1. Discretized Geometric Brownian Motion
We simulate the asset price $S_t$ on a discrete time grid $t_0, t_1, \ldots, t_K$ with $\Delta t = 1/252$ (daily steps).
Under the risk-neutral measure $\mathbb Q$, the dynamics are given by:

$$
S_{t_{k+1}} = S_{t_k} \exp\left( (r - \frac{1}{2}\sigma^2)\Delta t + \sigma\sqrt{\Delta t} Z_{k+1} \right)
$$

where $Z_{k+1} \sim \mathcal N(0,1)$ are i.i.d. standard normal variables.

### 10.2. Path-Dependent Barrier Logic
Unlike the simplified integral over observation dates, the Down-and-In Put feature requires tracking the **minimum** price over the entire path.
Let $\mathbf{S}^{(j)} = (S_{t_0}^{(j)}, \ldots, S_{t_K}^{(j)})$ denote the $j$-th simulated path.
The Knock-In event $E_{KI}$ is defined as:

$$
E_{KI}^{(j)} = \mathbb{1}_{\left\{ \min_{0 \le k \le K} S_{t_k}^{(j)} < B_{KI} \right\}}
$$

### 10.3. Payoff Function $\Phi(\mathbf{S})$
For each path, we evaluate the cashflows at observation dates $T_1, \ldots, T_n$:

1.  **Autocall**: If $S_{T_i} \ge A$, payoff is $N(1+C)e^{-rT_i}$. The path terminates.
2.  **Coupon**: If not autocalled but $S_{T_i} \ge B$, payoff is $NCe^{-rT_i}$.
3.  **Redemption**: At maturity $T_n$ (if not autocalled):
    *   If $S_{T_n} \ge B$: Pay $N$.
    *   If $S_{T_n} < B$ and $E_{KI}=1$ (Barrier Hit): Pay $N \times \frac{S_{T_n}}{S_0}$ (Capital Loss).
    *   If $S_{T_n} < B$ and $E_{KI}=0$ (Barrier Not Hit): Pay $N$ (Protected).

### 10.4. Monte Carlo Estimator

The core of the implementation is based on the Law of Large Numbers.
By simulating a large number of independent paths ($M$), we are essentially sampling from the joint distribution of asset prices.

The algorithm works as follows:

1.  **Concretize the Randomness**: We generate $M$ potential future scenarios (paths) for the stock price $S_t$. Each path is constructed using random normal increments $Z$.

    ![Monte Carlo Paths](I'm going to add a picture here)
    *Figure: Example of 20 simulated price paths relative to product barriers.*

2.  **Evaluate Path Performance**: For **each individual path** $j$, we calculate the specific cashflows generated by the product:
    *   Does it hit the Autocall barrier? $\to$ Payoff $CF_j = N(1+C)e^{-rT_{call}}$.
    *   Does it survive and pay coupons? $\to$ Payoff $CF_j = \sum \text{Coupons}$.
    *   Does it finish below the KI barrier? $\to$ Payoff $CF_j = \text{Reduced Capital}$.

    This results in a single present value number $PV_j$ for that specific path.

3.  **Average the Results**: Finally, we compute the arithmetic mean of these individual path values.

    $$
    \hat{V}_0 = \frac{1}{M} \sum_{j=1}^M PV_j
    $$

    This average $\hat{V}_0$ converges to the true theoretical price $V_0$ as we increase $M$.
    In our code, `compute_autocall_price` performs exactly this: it simulates `num_simulations` paths, computes the `payoffs` array (where each element corresponds to one path), and returns `np.mean(payoffs)`.
